<a href="https://colab.research.google.com/github/srigstinahf/Prediksi-RNN-LSTM-TFT-/blob/main/LSTM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

#Load & Preprocessing Data
data = pd.read_csv("sample_data/dataset seattle-weather.csv")
data.dropna(inplace=True)
data['date'] = pd.to_datetime(data['date'])

#Normalisasi temp_max
scaler = MinMaxScaler()
data['temp_max_scaled'] = scaler.fit_transform(data[['temp_max']])

#Siapkan data untuk training
def df_to_XY(series, window_size=10):
    X, y = [], []
    for i in range(window_size, len(series)):
        X.append(series[i-window_size:i])
        y.append(series[i])
    return np.array(X), np.array(y)

WINDOW = 10
series = data['temp_max_scaled'].values
X, y = df_to_XY(series, WINDOW)

#Split data
X_train, y_train = X[:800], y[:800]
X_val, y_val = X[800:1000], y[800:1000]
X_test, y_test = X[1000:], y[1000:]

#Reshape
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

#Build LSTM Model
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

#Train
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32)

#Plot Loss
hist = pd.DataFrame(history.history)
fig_loss = px.line(hist, y=['loss', 'val_loss'], title='Loss vs Val Loss', labels={'index': 'Epoch'})
fig_loss.show()

#Predict
train_pred = model.predict(X_train).flatten()
val_pred = model.predict(X_val).flatten()
test_pred = model.predict(X_test).flatten()

pred = np.concatenate([train_pred, val_pred, test_pred])

#Invers Skala ke Satuan Suhu Asli
actual_all = scaler.inverse_transform(series[WINDOW:].reshape(-1,1)).flatten()
pred_all = scaler.inverse_transform(pred.reshape(-1,1)).flatten()

#DataFrame
df_pred = pd.DataFrame({
    'actual': actual_all,
    'predicted': pred_all
})

#Plot Hasil Prediksi
fig_pred = px.line(df_pred[1000:], title="Test Results (Actual vs Predicted)",
                   labels={'index': 'Time', 'value': 'Temperature (°C)'})
fig_pred.add_scatter(y=df_pred['actual'][1000:], mode='lines', name='Actual', line=dict(color='blue'))
fig_pred.add_scatter(y=df_pred['predicted'][1000:], mode='lines', name='Predicted', line=dict(color='red'))
fig_pred.show()

#Evaluasi Metrik
mae = mean_absolute_error(df_pred['actual'], df_pred['predicted'])
mse = mean_squared_error(df_pred['actual'], df_pred['predicted'])
rmse = np.sqrt(mse)

actual_nonzero = df_pred['actual'] != 0
mape = np.mean(np.abs((df_pred['actual'][actual_nonzero] - df_pred['predicted'][actual_nonzero]) / df_pred['actual'][actual_nonzero])) * 100

print(f"\n📊 Evaluation on Test Set (LSTM):")
print(f"MAE  = {mae:.4f}")
print(f"RMSE = {rmse:.4f}")
print(f"MAPE = {mape:.2f}%")


Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 56ms/step - loss: 0.1038 - val_loss: 0.0281
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0183 - val_loss: 0.0137
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0149 - val_loss: 0.0156
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.0163 - val_loss: 0.0162
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.0134 - val_loss: 0.0162
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.0136 - val_loss: 0.0158
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0133 - val_loss: 0.0150
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0142 - val_loss: 0.0168
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0154 - val_loss: 0.0168
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0135 - val_loss: 0.0236
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0145 - val_loss: 0.0179
Epoch 12/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step



📊 Evaluation on Test Set (LSTM):
MAE  = 2.1610
RMSE = 2.7343
MAPE = 17.15%
